In [1]:
from dotenv import load_dotenv
from openai import Client

load_dotenv("D:\\python_de_learnert_data\\.env")

False

In [4]:
import json
import requests
from openai import OpenAI
client = OpenAI()

In [3]:
def get_length(your_word: str):
    """Function returns the length of the your_word string"""
    return len(your_word)

In [13]:
def run_conversation(content):
    messages = [{"role": "user", "content": content}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_length",
                "description": "Returns the length of the given your_word argument",
                "parameters":{
                    "type": "object",
                    "properties":{
                        "your_word":{
                            "type": "string",
                            "description": "word for which you require the length",
                        }
                    }
                }
            }
        }
    ]
    response = client.chat.completions.create(
        messages=messages,
        tools=tools,
        model="gpt-3.5-turbo",
        tool_choice="auto"
    ) 

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        messages.append(response_message)
        avbl_fun = {
            "get_length": get_length,
        }
        for tool_call in tool_calls:
            print(f"Name of function called: {tool_call.function.name}")
            print(f"Parameters of function called: {tool_call.function.arguments}")
            function_name = tool_call.function.name
            function_obj = avbl_fun[function_name] ## this is where the text function name is 
            # converted to function_obj 
            function_args = json.loads(tool_call.function.arguments)
            # argument values in the returned texts are converted to objects 
            function_response = function_obj(
                your_word=function_args['your_word']
            )
            print(f"Function response: {function_response}")
            # generated value is updated to the messages thread
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(function_response)
            }) 

    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        stream=True
    ) 
    return second_response 

In [14]:
question = "What is the length of word constantinople"

In [15]:
response = run_conversation(question)

Name of function called: get_length
Parameters of function called: {"your_word":"constantinople"}
Function response: 14


In [16]:
for chunk in response:
    print(chunk.choices[0].delta.content or " ", end = " ", flush=True)

  The  length  of  the  word  " constant in ople "  is   14 .   